# **Rejected Response Inference**

## **Config**

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"  # sesuaikan dengan GPU yang tersedia

seed = 42
DATASET_PATH    = "../datasets4545/cqa_rl_prompt_completion.jsonl"
CACHE_FOLDER    = "../model_cache"
SFT_MODEL_NAMES = {
    "Meta-Llama-3.1-8B": os.path.join("../sft/sft_output_Meta-Llama-3.1-8B", "final_checkpoint"),
    "Aya-23-8B":     os.path.join("../sft/sft_output_Aya-23-8B", "final_checkpoint"), 
    "SeaLLMs-v3-7B": os.path.join("../sft/sft_output_SeaLLMs-v3-7B", "final_checkpoint"),
    "SEA-LION-v3-8B":    os.path.join("../sft/sft_output_SEA-LION-v3-8B", "final_checkpoint"),
    "Sahabat-AI-8B":     os.path.join("../sft/sft_output_Sahabat-AI-8B", "final_checkpoint"),
}
BATCH_SIZE      = 32

## **Import Libraries**

In [2]:
import gc
import time
import json
import random
import torch
import pandas as pd
from tqdm.auto import tqdm
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    set_seed
)

pd.set_option("display.max_colwidth", None)

/raid/home/llmsosmed/rlaif/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## **Utility Functions**

In [3]:
def set_global_seed(s: int = seed):
    random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    set_seed(s)

def load_sft_model_and_tokenizer(model_path: str, tokenizer_path: str, hf_token: str = None):
    set_global_seed()
    tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_path,
        use_auth_token=hf_token,
        local_files_only=True
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        quantization_config=quant_config,
        low_cpu_mem_usage=True,
        use_auth_token=hf_token,
        local_files_only=True
    )
    model.config.use_cache = False
    model.eval()
    return tokenizer, model

def generate_sft_batch(tokenizer, model, prompts, max_new_tokens=256):
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1792
    ).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return [
        text[len(prompt):].split("\n", 1)[0]
        for text, prompt in zip(decoded, prompts)
    ]


In [4]:
set_global_seed()

## **Load Dataset**

In [5]:
df = pd.read_json(DATASET_PATH, lines=True)
print(f"Total examples: {len(df)}")
df.sample(3)

Total examples: 1524


,prompt,completion
782,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 50 Pada saat Peraturan Otoritas Jasa Keuangan ini mulai berlaku, Peraturan Otoritas Jasa Keuangan Nomor 30/POJK.04/2017 tentang Pembelian Kembali Saham yang Dikeluarkan oleh Perusahaan Terbuka (Lembaran Negara Republik Indonesia Tahun 2017 Nomor 130, Tambahan Lembaran Negara Republik Indonesia Nomor 6077), dicabut dan dinyatakan tidak berlaku. Pasal 51 Peraturan Otoritas Jasa Keuangan ini mulai berlaku pada tanggal diundangkan.\n\nQuestion: Apa saja cakupan dokumen menurut Pasal 50?\nAnswer:",Dokumen ini mencakup pencabutan dan penggantian Peraturan Otoritas Jasa Keuangan Nomor 30/POJK.04/2017 tentang Pembelian Kembali Saham yang Dikeluarkan oleh Perusahaan Terbuka.
76,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 17 (1) BPRS yang melanggar ketentuan sebagaimana dimaksud dalam Pasal 2 dan/atau Pasal 5 ayat (5) dikenai sanksi administratif berupa teguran tertulis sebanyak 2 (dua) kali dengan tenggang waktu 10 (sepuluh) hari kerja untuk setiap teguran tertulis. (2) Dalam hal BPRS telah dikenai sanksi sebagaimana dimaksud pada ayat (1) dan tidak memperbaiki atau tidak menyampaikan laporan dalam jangka waktu 10 (sepuluh) hari kerja setelah surat teguran terakhir, BPRS dikenai sanksi administratif berupa denda sebesar Rp5.000.000,00 (lima juta rupiah) dan: a. penurunan tingkat kesehatan BPRS sesuai dengan Peraturan Otoritas Jasa Keuangan mengenai sistem penilaian tingkat kesehatan bank pembiayaan rakyat syariah; dan/atau b. larangan sebagai pihak utama lembaga jasa keuangan sesuai dengan Peraturan Otoritas Jasa Keuangan mengenai penilaian kembali bagi pihak utama lembaga jasa keuangan.\n\nQuestion: Apa sanksi administratif bagi BPRS yang melanggar Pasal 2 dan/atau Pasal 5 ayat (5)?\nAnswer:",Sanksi administratif bagi BPRS yang melanggar Pasal 2 dan/atau Pasal 5 ayat (5) adalah teguran tertulis sebanyak 2 kali dengan tenggang waktu 10 hari kerja untuk setiap teguran tertulis.
1009,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 5 Fungsi sekretaris perusahaan melaksanakan tugas paling kurang: a. mengikuti perkembangan Pasar Modal khususnya peraturan perundang-undangan yang berlaku di bidang Pasar Modal; b. memberikan masukan kepada Direksi dan Dewan Komisaris Emiten atau Perusahaan Publik untuk mematuhi ketentuan peraturan perundang-undangan di bidang Pasar Modal; c. membantu Direksi dan Dewan Komisaris dalam pelaksanaan tata kelola perusahaan yang meliputi: 1. keterbukaan informasi kepada masyarakat, termasuk ketersediaan informasi pada Situs Web Emiten atau Perusahaan Publik; 2. penyampaian laporan kepada Otoritas Jasa Keuangan tepat waktu; 3. penyelenggaraan dan dokumentasi Rapat Umum Pemegang Saham; 4. penyelenggaraan dan dokumentasi rapat Direksi dan/atau Dewan Komisaris; dan 5. pelaksanaan program orientasi terhadap perusahaan bagi Direksi dan/atau Dewan Komisaris. d. sebagai penghubung antara Emiten atau Perusahaan Publik dengan pemegang saham Emiten atau Perusahaan Publik, Otoritas Jasa Keuangan, dan pemangku kepentingan lainnya.\n\nQuestion: Apa kewajiban yang ditetapkan terkait audit internal?\nAnswer:",Saya tidak tahu terkait kewajiban yang ditetapkan terkait audit internal.


## **Generate Rejected Responses & Build Preference Pairs**

In [6]:
# kita pakai satu SFT model (atau bisa loop jika mau multi-model)
for model_key, sft_path in SFT_MODEL_NAMES.items():
    print(f"\n▶ Generating rejected responses with SFT {model_key}")
    tokenizer_dir = os.path.join(CACHE_FOLDER, model_key)
    tokenizer, model = load_sft_model_and_tokenizer(
        sft_path, tokenizer_dir, hf_token=os.getenv("HF_TOKEN")
    )

    output_path = f"preference_{model_key}.jsonl"

    # warm-up…
    _ = model.generate(
        **tokenizer("Warm up", return_tensors="pt").to(model.device),
        max_new_tokens=1,
        pad_token_id=tokenizer.eos_token_id
    )

    records = []
    start = time.time()
    for i in tqdm(range(0, len(df), BATCH_SIZE), desc=f"Inferring {model_key}"):
        batch   = df.iloc[i : i + BATCH_SIZE]
        prompts = batch["prompt"].tolist()
        preds   = generate_sft_batch(tokenizer, model, prompts)

        for prompt, chosen, rejected in zip(prompts, batch["completion"], preds):
            records.append({
                "prompt":   prompt,
                "chosen":   chosen,
                "rejected": rejected
            })

    print(f"→ Inference time: {time.time()-start:.1f}s, total pairs: {len(records)}")

    # simpan raw preference file per model
    with open(output_path, "w", encoding="utf-8") as f:
        for rec in records:
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")
    print(f"✔️ Saved raw preference dataset to `{output_path}`")

    # cleaning duplicates
    df_pref = pd.read_json(output_path, lines=True)
    mask_same = df_pref["chosen"].str.strip() == df_pref["rejected"].str.strip()
    df_clean = df_pref[~mask_same].reset_index(drop=True)
    clean_path = output_path.replace(".jsonl", "_clean.jsonl")
    df_clean.to_json(clean_path, orient="records", lines=True, force_ascii=False)
    print(f"✔️ Cleaned preference dataset saved to `{clean_path}`")

    # tampil sampel
    display(df_clean.sample(5).reset_index(drop=True))

    # selesai untuk model ini
    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()


▶ Generating rejected responses with SFT Meta-Llama-3.1-8B
[2025-06-07 10:42:36,629] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
Inferring Meta-Llama-3.1-8B:   0%|          | 0/48 [00:00<?, ?it/s]/raid/home/llmsosmed/rlaif/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/raid/home/llmsosmed/rlaif/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Inferring Meta-Llama-3.1-8B: 100%|██████████| 48/48 [32:07<00:00, 40.16s/it]

→ Inference time: 1927.9s, total pairs: 1524
✔️ Saved raw preference dataset to `preference_Meta-Llama-3.1-8B.jsonl`
✔️ Cleaned preference dataset saved to `preference_Meta-Llama-3.1-8B_clean.jsonl`


prompt  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 39 Perusahaan Efek yang melakukan Transaksi Short Selling untuk kepentingan sendiri wajib mengikuti ketentuan sebagai berikut: a. sebelum melakukan Transaksi Short Selling, Perusahaan Efek: 1. telah membuka rekening terpisah untuk Transaksi Short Selling; 2. telah menyisihkan dana dan/atau Efek dalam rekening sebagaimana dimaksud pada angka 1 paling sedikit 50% (lima puluh persen) dari nilai Transaksi Short Selling sebagai aset yang disisihkan Perusahaan Efek untuk menutup risiko Transaksi Short Selling; dan 3. memastikan telah tersedia Efek pada saat penyelesaian Transaksi Short Selling meliputi: a) memiliki Efek lain yang dapat dikonversi atau ditukar menjadi Efek yang digunakan untuk penyelesaian Transaksi Short Selling; b) telah melaksanakan hak atas opsi atau waran untuk memperoleh Efek yang digunakan untuk penyelesaian Transaksi Short Selling; dan/atau c) telah melakukan perjanjian pinjam-meminjam Efek dalam Transaksi Short Selling dari dan/atau melalui pihak-pihak sebagaimana dimaksud dalam Pasal 35. Perjanjian pinjam-meminjam Efek dibuat dengan menggunakan kontrak sebagaimana dimaksud dalam Pasal 36 sampai dengan Pasal 38; b. pada saat Transaksi Short Selling pertama terjadi, nilai aset yang disisihkan sebagaimana dimaksud pada huruf a angka 2 ditambah dana yang diterima dari penjualan Efek melalui Transaksi Short Selling paling sedikit 150% (seratus lima puluh persen) dari nilai Transaksi Short Selling; c. nilai aset yang disisihkan ditambah dana yang diterima dari penjualan Efek melalui Transaksi Short Selling sebagaimana dimaksud pada huruf b dipelihara Perusahaan Efek paling sedikit 135% (seratus tiga puluh lima persen) dari nilai pasar wajar Efek pada Posisi Short; d. jika nilai aset yang disisihkan ditambah dana yang diterima dari penjualan Efek melalui Transaksi Short Selling sebagaimana dimaksud pada huruf b mengalami penurunan dan/atau nilai pasar wajar Efek dalam Posisi Short mengalami kenaikan sehingga nilai aset yang disisihkan ditambah dana yang diterima dari penjualan Efek melalui Transaksi Short Selling kurang dari: 1. 135% (seratus tiga puluh lima persen) dari nilai pasar wajar Efek dalam Posisi Short, Perusahaan Efek menambah aset yang disisihkan dan/atau membeli Efek yang ditransaksikan secara short selling paling lambat 3 (tiga) Hari Bursa, sehingga nilai aset yang disisihkan ditambah dana yang diterima dari penjualan Efek melalui Transaksi Short Selling tidak kurang dari 135% (seratus tiga puluh lima persen) dari nilai pasar wajar Efek dalam Posisi Short s


▶ Generating rejected responses with SFT Aya-23-8B


...

,prompt,chosen,rejected
0,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 12 (1) Perubahan Rencana Bisnis sebagaimana dimaksud dalam Pasal 11 ayat (1) hanya dapat dilakukan 1 (satu) kali. (2) Perubahan Rencana Bisnis sebagaimana dimaksud pada ayat (1) hanya dapat dilakukan paling lambat pada akhir bulan Juni tahun berjalan. (3) Ketentuan sebagaimana dimaksud pada ayat (1) dan ayat (2) dikecualikan bagi perubahan Rencana Bisnis yang dilaksanakan dalam rangka penugasan pemerintah pada LJKNB.\n\nQuestion: Berapa kali perubahan Rencana Bisnis dapat dilakukan menurut Pasal 12 ayat (1)?\nAnswer:",Ketentuan umum yang ditetapkan adalah hanya dapat dilakukan 1 (satu) kali.,1 (satu) kali.
1,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 1 Beberapa ketentuan dalam Peraturan Otoritas Jasa Keuangan Nomor 23/POJK.04/2016 tentang Reksa Dana Berbentuk Kontrak Investasi Kolektif (Lembaran Negara Republik Indonesia Tahun 2016 Nomor 109, Tambahan Lembaran Negara Republik Indonesia Nomor 5886) sebagaimana telah diubah dengan Peraturan Otoritas Jasa Keuangan Nomor 2/POJK.04/2020 tentang Perubahan Atas Peraturan Otoritas Jasa Keuangan Nomor 23/POJK.04/2016 tentang Reksa Dana Berbentuk Kontrak Investasi Kolektif (Lembaran Negara Republik Indonesia Tahun 2020 Nomor 6, Tambahan Lembaran Negara Republik Indonesia Nomor 6455) diubah sebagai berikut: 1. Di antara Pasal 15 dan Pasal 16 disisipkan 1 (satu) pasal, yakni Pasal 15A sehingga berbunyi sebagai berikut: Pasal 15A Pelaksanaan penghitungan Nilai Pasar Wajar, pengumuman, dan pelaporan Nilai Aktiva Bersih Reksa Dana berbasis Efek luar negeri berlaku ketentuan: a. batasan waktu kewajiban penghitungan dan penyampaian Nilai Pasar Wajar dari Efek dalam portofolio Reksa Dana oleh Manajer Investasi kepada\n\nQuestion: Apa objek yang diatur dalam Pasal 15A terkait Reksa Dana berbasis Efek luar negeri?\nAnswer:","Objek yang diatur meliputi pelaksanaan penghitungan Nilai Pasar Wajar, pengumuman, dan pelaporan Nilai Aktiva Bersih Reksa Dana berbasis Efek luar negeri.","Nilai Pasar Wajar, pengumuman, dan pelaporan Nilai Aktiva Bersih Reksa Dana berbasis Efek luar negeri. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak tahu terkait {question}. Saya tidak"
2,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nBAB VA PENILAIAN, PENETAPAN, DAN PELAPORAN KUALITAS PENDANAAN TRANSAKSI EFEK Pasal 29A (1) LPE wajib melakukan penilaian dan penetapan kualitas Pendanaan Transaksi Efek. (2) LPE wajib melaporkan hasil penilaian dan penetapan kualitas Pendanaan Transaksi Efek sebagaimana dimaksud pada ayat (1) kepada Otoritas Jasa Keuangan melalui SLIK. Pasal 29B (1) Otoritas Jasa Keuangan dapat melakukan asesmen lebih lanjut atas penilaian kualitas pendanaan yang dilaporkan LPE melalui pemeriksaan sumber informasi atau dokumen lainnya. (2) Dalam hal terjadi perbedaan penilaian kualitas Pendanaan Transaksi Ef


▶ Generating rejected responses with SFT Sahabat-AI-8B


Inferring Sahabat-AI-8B: 100%|██████████| 48/48 [9:12:22<00:00, 690.48s/it]  

→ Inference time: 33143.0s, total pairs: 1524
✔️ Saved raw preference dataset to `preference_Sahabat-AI-8B.jsonl`
✔️ Cleaned preference dataset saved to `preference_Sahabat-AI-8B_clean.jsonl`


,prompt,chosen,rejected
0,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 8 (1) Permohonan persetujuan prinsip pendirian BUS hasil Pemisahan sebagaimana dimaksud dalam Pasal 7 huruf a disampaikan oleh BUK yang memiliki UUS kepada Otoritas Jasa Keuangan dilampiri dengan: a. dokumen permohonan persetujuan prinsip sesuai dengan ketentuan peraturan perundang-undangan mengenai bank umum syariah; b. rancangan Pemisahan yang telah disetujui oleh Dewan Komisaris; c. konsep akta Pemisahan; d. rancangan perubahan anggaran dasar BUK yang memiliki UUS; e. laporan posisi keuangan intern UUS posisi bulan terakhir sebelum permohonan izin prinsip pendirian BUS hasil Pemisahan yang menunjukkan terpenuhinya modal minimum pendirian BUS sebagaimana dimaksud dalam Pasal 6 ayat (2) dan ayat (3) dan/atau bukti tambahan setoran modal dari calon pemegang saham; dan f. rencana tindak penyelesaian pelampauan batas maksimum penyaluran dana, jika ada.\n\nQuestion: Apa mekanisme pelaporan permohonan persetujuan prinsip pada Pasal 8?\nAnswer:",Saya tidak tahu terkait mekanisme pelaporan permohonan persetujuan prinsip pada Pasal 8.,"Permohonan persetujuan prinsip pendirian BUS hasil Pemisahan disampaikan oleh BUK yang memiliki UUS kepada Otoritas Jasa Keuangan dilampiri dengan dokumen permohonan persetujuan prinsip sesuai dengan ketentuan peraturan perundang-undangan mengenai bank umum syariah; rancangan Pemisahan yang telah disetujui oleh Dewan Komisaris; konsep akta Pemisahan; rancangan perubahan anggaran dasar BUK yang memiliki UUS; laporan posisi keuangan intern UUS posisi bulan terakhir sebelum permohonan izin prinsip pendirian BUS hasil Pemisahan yang menunjukkan terpenuhinya modal minimum pendirian BUS sebagaimana dimaksud dalam Pasal 6 ayat (2) dan ayat (3) dan/atau bukti tambahan setoran modal dari calon pemegang saham; dan rencana tindak penyelesaian pelampauan batas maksimum penyaluran dana, jika ada."
1,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nBAB I \nKETENTUAN UMUM \nPasal 1 \nDalam Peraturan Otoritas Jasa Keuangan ini yang dimaksud \ndengan: \n1.\nLembaga Jasa Keuangan yang selanjutnya disingkat LJK\nadalah lembaga yang melaksanakan kegiatan di sektor\nPerbankan, Pasar Modal, Perasuransian, Dana Pensiun,\nLembaga Pembiayaan, dan Lembaga Jasa Keuangan\nLainnya sebagaimana dimaksud dalam Undang-Undang\nmengenai Otoritas Jasa Keuangan.\n2.\nPerizinan Secara Elektronik di Sektor Jasa Keuangan\nadalah perizinan dalam satu kesatuan proses yang\ndimulai dari tahap permohonan sampai dengan tahap\npenyelesaian \nmelalui \n1 \n(satu) \nsistem \ndengan\nmemanfaatkan teknologi informasi.\n3.\nSistem \nPerizinan \nSecara \nElektronik \nadalah \nsistem\npelayanan perizinan satu pintu dengan memanfaatkan\nteknologi informasi.\n4.\nPemohon adalah LJK, orang perseorangan, atau badan\nusaha, baik yang berbentuk badan hukum maupun yang\ntidak berbentuk badan hukum, perusahaan, usaha\nbersama, asosiasi, atau kelompok yang terorganisasi\nyang telah atau akan melakukan kegiatan di sektor jasa\nkeuangan \nyang \nmengajukan \npermohonan \nproses\nperizinan melalui Sistem Perizinan Secara Elektronik.\n\nQuestion: Apa saja cakupan dokumen menurut Pasal 1?\nAnswer:","Dokumen ini mencakup definisi Lembaga Jasa Keuangan, Perizinan Secara Elektronik di Sektor Jasa Keuangan, Sistem Perizinan Secara Elektronik, dan Pemohon.",Saya tidak tahu terkait cakupan dokumen menurut Pasal 1.
2,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 1 Dalam Peraturan Otoritas Jasa Keuangan ini yang dimaksud dengan: 1. Lembaga Jasa Keuangan yang selanjutnya disingkat LJK ada

In [ ]:
import signal

os.kill(os.getpid(), signal.SIGTERM)